In [53]:
import tensorflow as tf
import numpy as np

mawi_features = np.load("mawi_features.pkl")
mawi_labels = np.load("mawi_labels.pkl") 
mawi_features = np.reshape(mawi_features, [-1,15,1])

def conv1D_Net1(input_1d):
    kernel_1d1 = tf.reshape(tf.truncated_normal([64,1,1], dtype=tf.float32), [64,1,1])
    conv1d = tf.nn.conv1d(input_1d, kernel_1d1, stride=1, padding='SAME')
    pool1 = tf.nn.pool(conv1d, [2], 'MAX', 'SAME', strides = [2])
    #pool1 = tf.nn.dropout(pool1, keep_prob)
     #Flattenning the output of ConvNets 
    flat_conv2 = tf.contrib.layers.flatten(pool1)

    datasize = flat_conv2.get_shape().as_list()[1]
    W_fc1 = tf.Variable(tf.truncated_normal([datasize, 128], stddev=0.02))
    b_fc1 = tf.Variable(tf.zeros([128]))
    h_fc1 = tf.nn.relu(tf.matmul(flat_conv2,W_fc1)+b_fc1)
    h_fc1 = tf.nn.dropout(h_fc1, keep_prob)

    # Output layer
    y_conv = tf.layers.dense(inputs=h_fc1, units=2)
    return y_conv
    
def batches(batch_size, features, labels):
    assert len(features) == len(labels)
    out_batches = []
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batches = [features[start_i:end_i], labels[start_i:end_i]] 
        out_batches.append(batches)
    return out_batches


In [54]:
n_classes = 2 
# Features and Labels
features = tf.placeholder(tf.float32, [None, 15, 1])
labels = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(dtype=tf.float32, name = "keep_prob")

# Learning rate 
learning_rate = 0.5

# Output of the model 
output = conv1D_Net1(features)

# Calculating cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = output, labels = labels))
# Optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(output,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Train or test with this batch size
batchSize =  128 
epochs = 200


init = tf.global_variables_initializer() 

with tf.Session() as sess:
    sess.run(init) 
    #train_batches = batches(batchSize, mawi_features, mawi_labels)
    
    for epoch in range(epochs): 
        #for batch_features, batch_labels in train_batches: 
        #    train_data = {features: batch_features, labels : batch_labels, keep_prob : 0.7}
        training_accuracy = sess.run(accuracy, feed_dict = {features: mawi_features, labels : mawi_labels, keep_prob : 1} )
            # Print status for every 10 epochs
        if epoch % 20 == 0:
            print('Epoch {:<3} - Training Accuracy: {}'.format(epoch,training_accuracy))

    # Save the variables to disk.
    save_model_path = './image_classification'
    # Save Model
    saver = tf.train.Saver()
    save_path = saver.save(sess, save_model_path)
    print("Model saved")

Epoch 0   - Training Accuracy: 0.7015054225921631
Epoch 20  - Training Accuracy: 0.30505064129829407
Epoch 40  - Training Accuracy: 0.6949493885040283
Epoch 60  - Training Accuracy: 0.6274467706680298
Epoch 80  - Training Accuracy: 0.6311953663825989
Epoch 100 - Training Accuracy: 0.3768231272697449
Epoch 120 - Training Accuracy: 0.6949259042739868
Epoch 140 - Training Accuracy: 0.33287540078163147
Epoch 160 - Training Accuracy: 0.37958890199661255
Epoch 180 - Training Accuracy: 0.6949493885040283
Model saved
